## Matrix factorization using ALS

In this notebook, I implemented matrix factorization using ALS method from library [implicit](https://github.com/benfred/implicit) to build a recommender for Instacart. The algorithm is based on [research by Yifan Hu et.al](http://yifanhu.net/PUB/cf.pdf).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
import os
sns.set()
%matplotlib inline

import implicit
import scipy.sparse as sparse

PROJ_ROOT = os.path.join(os.pardir)

# Table of Contents

* [1. Load Datasets](#1.-Load-Datasets)
* [2. Data Preprocessing](#2.-Data-Preprocessing)
* [3. Evaluation Metric: Recall@K](#3.-Evaluation-Metric:-Recall@K)
* [4. Benchmark Model: Non-personalized Popularity Model](#4.-Benchmark-Model:-Non-personalized-Popularity-Model)
* [5. Matrix Factorization Model](#5.-Matrix-Factorization-Model)
    * [5.1 Try Default Model ](#5.1-Try-Default-Model)
    * [5.2 Tuning Model](#5.2-Tuning-Model)

### 1. Load Datasets

In [2]:
orders = pd.read_csv('../data/raw/orders.csv')
order_products = pd.read_csv('../data/raw/order_products__prior.csv')
products = pd.read_csv('../data/raw/products.csv')

In [3]:
# fill na with 0 
orders.fillna(0, inplace=True)

In [4]:
# merge orders and order_products on order_id
orders_df = order_products[['order_id', 'product_id']].merge(orders[['order_id', 'user_id', 'order_number']])

In [5]:
orders_df.head()

,order_id,product_id,user_id,order_number
0,2,33120,202279,3
1,2,28985,202279,3
2,2,9327,202279,3
3,2,45918,202279,3
4,2,30035,202279,3


### 2. Data Preprocessing

In [6]:
def prior_latest(df):
    '''Label each order with prior or latest'''
    max_row = df['order_number'].max()
    labels = np.where(df['order_number'] == max_row,
                     'latest', 
                     'prior')
    return pd.DataFrame(labels, index=df.index)

def split_train_test_set(df):
    '''
    df is merged order-products dataset containing order_id, product_id, user_id and order_number,
    Split df into training and test data where prior orders are training data and
    most recent orders are test data
    '''
    df['set'] = df.groupby('user_id').apply(prior_latest)
    training_set = df[df.set == 'prior'].drop('set', axis=1)
    test_set = df[df.set == 'latest'].drop('set', axis=1)
    
    # sanity check
    assert len(training_set) + len(test_set) == len(df)
    assert training_set.order_id.nunique() + test_set.order_id.nunique() == df.order_id.nunique()
    
    return training_set, test_set

def make_test_data(test_set):
    '''
    convert test_set to a dataframe in the form of only two columns, first column is user_id, 
    second column is a list of products purchased by the user in their most recent order
    '''
    test_data = test_set.groupby('user_id').product_id.apply(list).reset_index().rename(
                columns={'product_id': 'products'})
    return test_data


def get_user_item_matrix(training_set):
    '''
    make sparse user item matrix from user_product_quantity_df
    rows represent users, cols represent items, values in the matrix represent the number of purchases 
    '''
    user_product_quantity_df = training_set.drop('order_number', axis=1).groupby(['user_id', 'product_id']).count(
                                ).reset_index().rename(columns={'order_id':'quantity'})
    
    users = list(np.sort(user_product_quantity_df.user_id.unique()))
    items = list(user_product_quantity_df.product_id.unique())
    quantity = list(user_product_quantity_df.quantity)
    
    rows = user_product_quantity_df.user_id.astype('category', CategoricalDtype=users).cat.codes
    cols = user_product_quantity_df.product_id.astype('category', CategoricalDtype=items).cat.codes
    user_item_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(users), len(items)))
    
    return user_item_sparse

def get_prod_names(product_ids, df=products):
    '''generate product names from a list of product ids'''
    return df[df.product_id.isin(product_ids)][['product_id', 'product_name']]

In [7]:
# split dataset into training and test set
training_set, test_set = split_train_test_set(orders_df)

# make test data in the form we want
test_data = make_test_data(test_set)

# create user-item interaction matrix
user_item_sparse = get_user_item_matrix(training_set)

In [8]:
# user_item_sparse is our training data
user_item_sparse

<206209x49652 sparse matrix of type '<class 'numpy.int64'>'
	with 12422758 stored elements in Compressed Sparse Row format>

In [9]:
# Calculate sparsity ratio
def sparsity(sparse_matrix):
    ''' Calculate the sparsity ratio of a sparse matrix'''
    matrix_size = sparse_matrix.shape[0] * sparse_matrix.shape[1]
    num_non_zeros = len(sparse_matrix.nonzero()[0])
    sparsity = 100 * (1 - num_non_zeros/matrix_size)
    return sparsity 

In [10]:
sparsity(user_item_sparse)

99.87866847332864

In [11]:
test_data.head()

,user_id,products
0,1,"[196, 46149, 39657, 38928, 25133, 10258, 35951..."
1,2,"[24852, 16589, 1559, 19156, 18523, 22825, 2741..."
2,3,"[39190, 18599, 23650, 21903, 47766, 24810]"
3,4,"[26576, 25623, 21573]"
4,5,"[27344, 24535, 43693, 40706, 16168, 21413, 139..."


- **training_data** is a sparse matrix with 99.88% zero interactions. 
- **test_data** is the hold out data for us to evaluate the model which shows a list of products purchased by each user in their most recent order with Instacart. 

## 3. Evaluation Metric: Recall@K

Evaluation metric I choose for the recommender of this case is recall@k. Since for our case, the feedback is implicit, there's not explicit rating scores. It is more appropriate to use classification accuracy metrics. Precision as we know represents the proportion of recommended items that actually bought by the users. It is not that appropriate since we do not know users' reaction to the recommended items. Besides, precision will give us the same result for two users who both bought 2 items from the recommendation list with 10 items, but one of them bought a total of 5 items while the other only bought a total of those 2 items. While the performance of the recommender should not be the same. Recall is more appropriate which represents the proportaion of acutal bought items that are from the recommended list. 

In [12]:
def recall_at_k(rec, act):
    '''
    calculate recall from a list of predicted products and a list of actual purchased products
    '''
    return len(set(rec).intersection(set(act)))/len(set(act)) 

def mean_recall_at_k_rec(rec_k, test_data):
    '''
    Calculate mean recall score for our recommender given the recommended output and the test data
    '''
    score = []
    for i in range(len(test_data)):
        score.append(recall_at_k(rec_k.products[i], test_data.products[i]))
    return np.mean(score)

In [13]:
# create a dictionary to associate products' position index in the sparse matrix with product_id
prod_dict = dict(enumerate(training_set["product_id"].astype('category').cat.categories))

def mean_recall_at_k_rec(model, test_data, user_item, k, p_dict):
    '''
    Calculate mean recall score for our recommender given the model, training data and the test data
    '''
    
    test_data_rec = test_data.copy()
    rec_list = []
    recall_score = []
    
    for i in list(test_data['user_id']):
        rec_id = [j[0] for j in model.recommend(i-1, user_item, N=k)]
        rec_prod = [p_dict[id] for id in rec_id]
        rec_list.append(rec_prod)
        recall_score.append(recall_at_k(rec=rec_prod, act=test_data.products[i-1]))
    test_data_rec['rec'] = rec_list
    test_data_rec['score'] = recall_score
                
    return np.mean(test_data_rec['score'])

From the EDA part, we know that reorder ratio is quite high. Since recommender is not only about recommending items that users have already purchased before, but also about recommending items that users may be interested in but are unaware of them. Therefore, I would like to build another evaluation metric, still using recall, but eliminating reordered items from test data to check the proportion of first time ordered products that are from the recommended products. 

In [14]:
def mean_recall_at_k_rec_new(model, test_data, user_item, k, p_dict):
    '''
    Calculate mean recall score for our recommender given the model, training data and the test data 
    with reordered products eliminated
    '''
    purchased = training_set.groupby('user_id').product_id.apply(list).reset_index().rename(
                columns={'product_id': 'purchased'})
    
    test_data_rec = test_data.copy()
    rec_list = []
    recall_score = []
    
    for i in list(test_data['user_id']):
        rec_id = [j[0] for j in model.recommend(i-1, user_item, N=k)]
        rec_prod = [p_dict[id] for id in rec_id]
        rec_list.append(rec_prod)
        
        # get rid of reordered products
        new = list(set(test_data.products[i-1]) - set(purchased.purchased[i-1]))
    # deal with the situation when all products in users's most recent order are reordered
        if not new:
            recall_score.append(0)
        else:
            recall_score.append(recall_at_k(rec_prod, new))
            
    test_data_rec['rec'] = rec_list
    test_data_rec['score'] = recall_score
            
    return np.mean(test_data_rec['score'])

## 4. Benchmark Model: Non-personalized Popularity Model

In [15]:
def most_popular_k(k, df=training_set):
    '''
    get the most popular k items to be our benchmark recommender
    '''
    return df.product_id.value_counts().head(k).keys().tolist()

def mean_recall_at_k_pop(top_k, test_data):
    '''
    Calculate mean recall score for the benchmark recommender given the top_k list and the test data
    '''
    score = []
    for i in range(len(test_data)):
         score.append(recall_at_k(top_k, test_data.products[i]))
    return np.mean(score)

def mean_recall_at_k_pop_new(top_k, test_data):
    '''
    Calculate mean recall score for the benchmark recommender given the top_k list and the test data with 
    reordered items eliminated
    '''
    purchased = training_set.groupby('user_id').product_id.apply(list).reset_index().rename(
                columns={'product_id': 'purchased'})
    score = []
    for i in range(len(test_data)):
        # get rid of reordered products
        new = list(set(test_data.products[i]) - set(purchased.purchased[i]))
    # deal with the situation when all products in users's most recent order are reordered
        if not new:
            score.append(0)
        else:
            score.append(recall_at_k(top_k, new))
    return np.mean(score)

In [16]:
k = [10, 20, 50]

for i in k:
    top_k = most_popular_k(i, df=training_set)
    print('recall@{0} for popularity model is: {1}'.format(i, mean_recall_at_k_pop(top_k, test_data)))
    print('recall@{0} for popularity model without reordered products is: {1}'.format(i, 
                                                            mean_recall_at_k_pop_new(top_k, test_data)))

recall@10 for popularity model is: 0.0699413179973539
recall@10 for popularity model without reordered products is: 0.027280349700253143
recall@20 for popularity model is: 0.09588023492718917
recall@20 for popularity model without reordered products is: 0.043705006414388715
recall@50 for popularity model is: 0.15431186904512134
recall@50 for popularity model without reordered products is: 0.08048940655391035


- We can see that recall score increases with the number of products recommended increases. It makes sense cause as we recommend more, more acutually purchased items will fall into the recommended list. 
- The popularity model is not performing that well in recommending people to try new products they've never bought before. This is because as we know from the EDA the top popular products are similar as those most reordered items. Thus when we get rid of the reordered items from the test data, the interaction of those two lists would shrink. 

#### Example: user_id == 100

In [17]:
print('User_id 100 actually bought: ')
get_prod_names(test_data.products[99])

User_id 100 actually bought: 


,product_id,product_name
21136,21137,Organic Strawberries
21615,21616,Organic Baby Arugula
24851,24852,Banana
26368,26369,Organic Roma Tomato
27343,27344,Uncured Genoa Salami
38546,38547,Bubblegum Flavor Natural Chewing Gum
38688,38689,Organic Reduced Fat Milk
48627,48628,Organic Whole Wheat Bread


In [18]:
print('User_id 100 got recommended: ')
get_prod_names(most_popular_k(10, df=training_set))

User_id 100 got recommended: 


,product_id,product_name
13175,13176,Bag of Organic Bananas
16796,16797,Strawberries
21136,21137,Organic Strawberries
21902,21903,Organic Baby Spinach
24851,24852,Banana
26208,26209,Limes
27844,27845,Organic Whole Milk
47208,47209,Organic Hass Avocado
47625,47626,Large Lemon
47765,47766,Organic Avocado


For the customer with user_id = 100, we see that he/she bought two of the products from the recommended list: organic straberries and banana. 

## 5. Matrix Factorization Model

### 5.1 Try Default Model 

In [19]:
# input of implicit als model is item_users matrix with confidence being the value
item_user_sparse = user_item_sparse.T.tocsr()
item_users = 10 * item_user_sparse

# initiate and fit the model with default parameter settings
als = implicit.als.AlternatingLeastSquares()

als.fit(item_users)

100%|██████████| 15.0/15 [01:00<00:00,  3.90s/it]


In [20]:
# check the recall score including reordered items
mean_recall_at_k_rec(model=als, test_data=test_data, user_item=user_item_sparse, k=10, p_dict=prod_dict)

0.021157856107798727

In [21]:
# check the recall score without reordered items
mean_recall_at_k_rec_new(model=als, test_data=test_data, user_item=user_item_sparse, k=10, p_dict=prod_dict)

0.04400496554150407

The recall score is less than the popularity model if reordered items were included, but it's better than the popularity model if reordere items were excluded. The Collaborative Filtering model using matrix factorization is better at recommending new products to customers than the benchmark model.

### 5.2 Tuning Model

In [22]:
# Build train and validation sets within the training set
train_cv, val_cv = split_train_test_set(training_set)

val_data = make_test_data(val_cv)

user_item_cv = get_user_item_matrix(train_cv)

In [23]:
# create another dictionary for train_cv
prod_dict_cv = dict(enumerate(train_cv["product_id"].astype('category').cat.categories))

In [133]:
# It takes several hours to do the grid search
alphas = [10, 15]
factors = [30, 40, 50]
regparams = [0.01, 0.1, 1.0]
iterations = [25, 50]

best_recall_score = 0
best_alpha = 0
best_factors = 0
best_reg = 0
best_iterations = 0


for a in alphas:
    for f in factors:
        for r in regparams:
            for i in iterations:
               
                # get item_user_matrix matrix 
                item_user_cv = user_item_cv.T.tocsr()
                
                # create ALS model
                als = implicit.als.AlternatingLeastSquares(factors=f, regularization=r, iterations=i)
                
                # fit model 
                als.fit(a * item_user_cv)
                
                
                score = mean_recall_at_k_rec(model=als, test_data=val_data, user_item=user_item_cv, k=10, 
                                            p_dict=prod_dict_cv)

                print("alpha: ", a, "factors: ", f, "RegPram: ", r, "Iteration: ", i)
                
                if score > best_recall_score:
                    best_recall_score = score
                    best_alpha = a
                    best_factors = f
                    best_reg = r
                    best_iterations = i
print("Best alpha: ", best_alpha, "Best factors: ", best_factors, "Best RegPram: ", best_reg, "Best Iteration: ", 
     best_iterations, 'Best recall: ', best_recall_score)

100%|██████████| 25.0/25 [01:06<00:00,  2.41s/it]


alpha:  10 factors:  30 RegPram:  0.01 Iteration:  25


100%|██████████| 50.0/50 [02:12<00:00,  2.88s/it]


alpha:  10 factors:  30 RegPram:  0.01 Iteration:  50


100%|██████████| 25.0/25 [00:58<00:00,  2.37s/it]


...

alpha:  15 factors:  50 RegPram:  0.1 Iteration:  50


100%|██████████| 25.0/25 [01:04<00:00,  2.68s/it]


alpha:  15 factors:  50 RegPram:  1.0 Iteration:  25


100%|██████████| 50.0/50 [02:10<00:00,  2.65s/it]


alpha:  15 factors:  50 RegPram:  1.0 Iteration:  50
Best alpha:  10 Best factors:  50 Best RegPram:  1.0 Best Iteration:  50 Best recall:  0.022253164111646982


In [24]:
best_alpha = 10
best_factors = 50
best_reg = 1.0
best_iterations = 50

In [25]:
# test tuned model with test_data
model = implicit.als.AlternatingLeastSquares(factors=best_factors, regularization=best_reg, iterations=best_iterations)
model.fit(best_alpha * item_user_sparse)

100%|██████████| 50.0/50 [02:21<00:00,  2.87s/it]


In [28]:
N = [10, 20, 50]

for i in N:
    print('recall@{0} including reordered products is: {1}'.format(i,
        mean_recall_at_k_rec(model=model, test_data=test_data, user_item=user_item_sparse, k=i, p_dict=prod_dict)))
    print('recall@{0} without reordered products is: {1}'.format(i, 
        mean_recall_at_k_rec_new(model=model, test_data=test_data, user_item=user_item_sparse, k=i, p_dict=prod_dict)))

recall@10 including reordered products is: 0.021108589265309844
recall@10 without reordered products is: 0.04362829431987226
recall@20 including reordered products is: 0.03470740114728848
recall@20 without reordered products is: 0.07196112895601287
recall@50 including reordered products is: 0.06281738009812805
recall@50 without reordered products is: 0.13033247828445535


By comparison, all recall@k with reordered products are lower than the benchmark popularity model. While the ALS model is performing better after we get rid of all the reordered products. So it's doing a better job in recommending new stuff to consumers. 

#### Example: user_id == 100

In [29]:
# recommend for a user, result is a list of (item, score) tuples
model.recommend(99, user_item_sparse)

[(47731, 0.68300116),
 (49646, 0.59405833),
 (47591, 0.5931705),
 (16787, 0.5908689),
 (21890, 0.5428823),
 (34948, 0.5374364),
 (28187, 0.53596514),
 (26194, 0.52375853),
 (13167, 0.49394482),
 (9071, 0.49373534)]

In [30]:
# check the product names from the above recommemdation result
prod_ids = [i[0] for i in model.recommend(99, user_item_sparse)]
products[products.product_id.isin([prod_dict[id] for id in prod_ids])][['product_id', 'product_name']]

,product_id,product_name
9075,9076,Blueberries
13175,13176,Bag of Organic Bananas
16796,16797,Strawberries
21902,21903,Organic Baby Spinach
26208,26209,Limes
28203,28204,Organic Fuji Apple
34968,34969,Red Vine Tomato
47625,47626,Large Lemon
47765,47766,Organic Avocado
49682,49683,Cucumber Kirby


In [31]:
# what user_id 100 actually bought?
print('User_id 100 actually bought: ')
get_prod_names(test_data.products[99])

User_id 100 actually bought: 


,product_id,product_name
21136,21137,Organic Strawberries
21615,21616,Organic Baby Arugula
24851,24852,Banana
26368,26369,Organic Roma Tomato
27343,27344,Uncured Genoa Salami
38546,38547,Bubblegum Flavor Natural Chewing Gum
38688,38689,Organic Reduced Fat Milk
48627,48628,Organic Whole Wheat Bread
